# HDL in MVP

## Reference panel

`download_1000G()` in `bigsnpr`

Including 2490 (mostly unrelated) individuals and ~1.7M SNPs in common with either HapMap3 or the UK Biobank.

## Base data: summary Statistics from MVP

Posterior betas for traits HDL.

## Target data: UK biobank

covariates, phenotype related to HDL and genotypes of 2000 individuals `UKB.QC.fam`. 

## Model

The model I used to estimate effect size is auto mdoel. It runs the algorithm for 30 different p (the proportion of causal variants) values range from 10e-4 to 0.9, and heritability from LD score regression as initial value.



## Data preparation

Use `awk` select columns in phenotypes file saved to traits file `UKB.hdl.cov` and covaraites file `UKB.ind.cov`.

## Step 1: common snps

In [5]:
sos run ldpred.ipynb extract_snp -v1 \
    --outpath res-data \
    --testpath ukbiobank \
    --ref_bed 1000G/1000G.EUR.bed \
    --test_bed ukbiobank/UKB.bed \
    --ref_snp 1000G/1000G.QC.snplist \
    --test_snp ukbiobank/UKB.QC.snplist \
    --summstats_file mvpdata/pos_sumstats_hdl.rds \
    --stat_snp mvpdata/pos_sumstats_hdl.snplist

PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ukbiobank/UKB.QC.log.
Options in effect:
  --bfile ukbiobank/UKB
  --geno 0.01
  --maf 0.01
  --mind 0.01
  --out ukbiobank/UKB.QC
  --write-snplist

8192 MB RAM detected; reserving 4096 MB for main workspace.
529024 variants loaded from .bim file.
366732 people (168262 males, 198470 females) loaded from .fam.
282 people removed due to missing genotype data (--mind).
IDs written to ukbiobank/UKB.QC.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 366450 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.997994.
81 variants remove

: 1

In [7]:
sos run ldpred.ipynb common_snp \
    --outpath res-data \
    --testpath ukbiobank \
    --stat_snp mvpdata/pos_sumstats_hdl.snplist \
    --ref_snp 1000G/1000G.QC.snplist \
    --test_snp ukbiobank/UKB.QC.snplist \
    --summstats_file mvpdata/pos_sumstats_hdl.rds \
    --sub_stats mvpdata/pos_sumstats_hdl.SUB.rds

INFO: Running common_snp: 2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
[1] "There are  409486  common SNPs."
INFO: common_snp is completed.
INFO: common_snp output:   mvpdata/pos_sumstats_hdl.SUB.rds res-data/common.snplist
INFO: Workflow common_snp (ID=w5eab50cc3f321432) is executed successfully with 1 completed step.


: 1

## Step 2: subsetting reference panel

In [8]:
sos run ldpred.ipynb subsets \
    --outpath res-data \
    --testpath ukbiobank \
    --bed_file 1000G/1000G.EUR.bed \
    --fam_file 1000G/1000G.EUR.fam \
    --snp_file res-data/common.snplist \
    --sub_bedfile 1000G/1000G.SUB.bed

INFO: Running subsets: 
PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 1000G/1000G.SUB.log.
Options in effect:
  --bfile 1000G/1000G.EUR
  --extract res-data/common.snplist
  --keep 1000G/1000G.EUR.fam
  --make-bed
  --out 1000G/1000G.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
1664852 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 409486 variants remaining.
--keep: 503 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 503 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
409486 variants and 503 people pass filters and QC.
Note: No phenotypes

: 1

Totally 31566 varients

    ./plink \
        --bfile 1000G/1000G.EUR \
        --keep 1000G/1000G.EUR.fam \
        --extract res-data/common.snplist \
        --make-bed \
        --out 1000G/1000G.SUB


## Step 3: SNP Matching


In [10]:
sos run ldpred.ipynb data_load \
    --outpath res-data \
    --testpath ukbiobank \
    --ref_bfile 1000G/1000G.SUB.bed \
    --ref_file 1000G/1000G.SUB.rds \
    --summstats_file mvpdata/pos_sumstats_hdl.SUB.rds \
    --n_eff 200000 \
    --test_snplist UKB.SUB.snplist

INFO: Running data_load_10: 04l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
[1] "/Users/zhangmengyu/Documents/bioworkflows/ldpred/1000G/1000G.SUB.rds"
INFO: data_load_10 is completed.
INFO: data_load_10 output:   res-data/SumStats.rds
INFO: Running data_load_20: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
409,486 variants to be matched.
21,954 ambiguous SNPs have been removed.
387,532 variants have been matched; 0 were flipped and 211,410 were reversed.
INFO: data_load_20 is completed.
INFO: data_load_20 output:   res-data/MatchedSnp.RData ukbiobank/UKB.SUB.snplist
INFO: Workflow data_load (ID=waa7d57a2e86f0332)

: 1

## Step 4: Quality control (or do not)

Greatly drop variants. 

In [11]:
sos run ldpred.ipynb QControl \
    --qc_in res-data/MatchedSnp.RData \
    --outpath res-data \
    --testpath ukbiobank \
    --test_snplist UKB.QC.SUB.snplist

INFO: Running QControl: [?2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Saving 7 x 7 in image
[1] "387086 over 387532 were removed in Quality Control."
INFO: QControl is completed.
INFO: QControl output:   res-da

: 1

## Step 5: subsetting target data

In [12]:
sos run ldpred.ipynb subsets \
    --outpath res-data \
    --testpath ukbiobank \
    --bed_file ukbiobank/UKB.bed \
    --fam_file ukbiobank/UKB.QC.fam \
    --snp_file ukbiobank/UKB.QC.SUB.snplist \
    --sub_bedfile ukbiobank/UKB.SUB.bed

INFO: Running subsets: 
PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ukbiobank/UKB.SUB.log.
Options in effect:
  --bfile ukbiobank/UKB
  --extract ukbiobank/UKB.QC.SUB.snplist
  --keep ukbiobank/UKB.QC.fam
  --make-bed
  --out ukbiobank/UKB.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
529024 variants loaded from .bim file.
366732 people (168262 males, 198470 females) loaded from .fam.
--extract: 446 variants remaining.
--keep: 2000 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2000 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.997422.


: 1

## Step 6: Calculate LD matrix and correlation

In [13]:
sos run ldpred.ipynb LD \
    --outpath res-data \
    --testpath ukbiobank \
    --ld_in res-data/QcMatchedSnp.Rdata

INFO: Running LD: 2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
INFO: LD is completed.
INFO: LD output:   res-data/LdMatrix.Rdata
INFO: Workflow LD (ID=w10cce9261a8a96e3) is executed successfully with 1 completed step.


: 1

## Step 7: Estimate posterior effect sizes and PRS

In [14]:
sos run ldpred.ipynb load_testdata+inf_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --inf_in res-data/LdMatrix.Rdata \
    --test_bfile ukbiobank/UKB.SUB.bed \
    --test_file ukbiobank/UKB.SUB.rds

INFO: Running load_testdata: 4l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
[1] "/Users/zhangmengyu/Documents/bioworkflows/ldpred/ukbiobank/UKB.SUB.rds"
INFO: load_testdata is completed.
INFO: Running inf_prs: 
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
[1] "finish imputation"
446 variants to be matched.
0 ambiguous SNPs have been removed.
446 variants have been matched; 0 were flipped and 446 were reversed.
[1] "finish estimate beta"
INFO: inf_prs is completed.
INFO: inf_prs output:   res-data/InfPred.Rdata
INFO: Workflow load_testdata+inf_prs (ID=we219d29404b7b0ae) is executed successfully with 2 completed ste

: 1

In [15]:
sos run ldpred.ipynb grid_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --grid_in res-data/LdMatrix.Rdata \
        --test_bfile ukbiobank/UKB.SUB.bed \
    --test_file ukbiobank/UKB.SUB.rds

INFO: Running grid_prs: [?2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
446 variants to be matched.
0 ambiguous SNPs have been removed.
446 variants have been matched; 0 were flipped and 446 were reversed.
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
INFO: grid_prs is completed.
INFO: grid_prs output:   res-data/GridPred.Rdata
INFO: Workflow grid_prs (ID=we4624375c0bb8c52) is executed successfully with 1 completed step.


: 1

In [16]:
sos run ldpred.ipynb auto_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --auto_in res-data/LdMatrix.Rdata \
    --test_bfile ukbiobank/UKB.SUB.bed \
    --test_file ukbiobank/UKB.SUB.rds

INFO: Running auto_prs: [?2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
446 variants to be matched.
0 ambiguous SNPs have been removed.
446 variants have been matched; 0 were flipped and 446 were reversed.
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
During startup - During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Saving

: 1

## Step 8: predict phenotypes

In [17]:
sos run ldpred.ipynb null_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --response 1

INFO: Running null_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 NULL model 0.0397 0.943
null device 
          1 
INFO: null_phenopred is completed.
INFO: null_phenopred output:   res-data/summary/NullSummary.pdf res-data/model/NullModel.Rdata
INFO: Workflow null_phenopred (ID=w02bc229370001ec3) is executed successfully with 1 completed step.


: 1

In [22]:
sos run ldpred.ipynb inf_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --inf_file res-data/InfPred.Rdata \
    --response 1

INFO: Running inf_phenopred: 4l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model     R2   MSE
  <chr>  <dbl> <dbl>
1 Model 0.0417 0.946
null device 
          1 
INFO: inf_phenopred is completed.
INFO: inf_phenopred output:   res-data/summary/InfSummary.pdf res-data/model/InfModel.Rdata
INFO: Workflow inf_phenopred (ID=w37cfdf1595ffd745) is executed successfully with 1 completed step.
bash: --response: command not found


: 1

In [26]:
sos run ldpred.ipynb grid_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --grid_file res-data/GridPred.Rdata \
    --response 1

INFO: Running grid_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
Saving 7 x 7 in image
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 Grid model 0.0513 0.937
pdf 
  2 
INFO: grid_phenopred is completed.
INFO: grid_phenopred output:   res-data/summary/GridSummary.pdf res-data/model/GridModel.Rdata... (3 items)
INFO: Workflow grid_phenopred (ID=wc3a7c7e254bac02a) is executed successfully with 1 completed step.


: 1

In [27]:
sos run ldpred.ipynb auto_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --auto_file res-data/AutoPred.Rdata \
    --response 1

INFO: Running auto_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
There were 50 or more warnings (use warnings() to see the first 50)
# A tibble: 1 x 3
  model          R2   MSE
  <chr>       <dbl> <dbl>
1 Auto model 0.0468 0.943
null device 
          1 
INFO: auto_phenopred is completed.
INFO: auto_phenopred output:   res-data/summary/AutoSummary.pdf res-data/model/AutoModel.Rdata
INFO: Workflow auto_phenopred (ID=wa926c4976d2463a6) is executed successfully with 1 completed step.


: 1

# Results

Following table shows adjusted R squared of phenotype prediction model. QC is quality control in step 4.

|   betas   | QC? |   Null  |   Inf   |   Grid  |   Auto  |
|:---------:|:---:|:-------:|:-------:|:-------:|:-------:|
|  Original | Yes | 0.03486 | 0.03549 | 0.03431 | 0.03564 |
| Posterior | Yes | 0.03486 | 0.03628 | 0.04595 | 0.04142 |
| Posterior |  No | 0.03486 | 0.03516 |    -$^{*}$    |    -$^*$    |

$*$: Take long very long time to get results (4hrs+)